In [1]:
%load_ext autoreload
%autoreload 2

import json
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from multiprocessing.pool import ThreadPool
from multiprocessing import Process, Manager, Pool
from collections import Counter
import sys

sys.path.append("../")
from utils.parse_arxiv import *

/data/zyang/miniconda3/envs/pt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
chunks = pd.read_json(
    "../data/arxiv-metadata-oai-snapshot.json", lines=True, chunksize=50000
)

In [3]:
# manager = Manager()

# author_dict = manager.dict()
# cat_dict = manager.dict()
# year_dict = manager.dict()
author_dict = {}
cat_dict = {}
journal_dict = {}
year_dict = {}

counter = 0
for chunk in chunks:
    df = chunk
    (
        intermediate_authors_stats,
        intermediate_cat_stats,
        intermediate_jou_stats,
        intermediate_year_stats,
    ) = gather_stats(df)
    author_dict = merge_add_dict(author_dict, intermediate_authors_stats)
    cat_dict = merge_add_dict(cat_dict, intermediate_cat_stats)
    journal_dict = merge_add_dict(journal_dict, intermediate_jou_stats)
    year_dict = merge_add_dict(year_dict, intermediate_year_stats)

    counter += len(df)
    print("processed:", counter)
    print("aut stats => ", len(author_dict))
    print("cat stats => ", len(cat_dict))
    print("journal stats => ", len(journal_dict))
    print("year stats => ", len(year_dict))
    # break

processed: 50000
aut stats =>  100892
cat stats =>  3869
journal stats =>  25312
year stats =>  2639
processed: 100000
aut stats =>  172824
cat stats =>  6070
journal stats =>  50140
year stats =>  3260
processed: 150000
aut stats =>  236791
cat stats =>  8414
journal stats =>  75579
year stats =>  3637
processed: 200000
aut stats =>  297452
cat stats =>  11079
journal stats =>  99626
year stats =>  3871
processed: 250000
aut stats =>  354072
cat stats =>  13849
journal stats =>  122164
year stats =>  4050
processed: 300000
aut stats =>  408798
cat stats =>  16471
journal stats =>  141753
year stats =>  4197
processed: 350000
aut stats =>  462450
cat stats =>  18807
journal stats =>  160487
year stats =>  4317
processed: 400000
aut stats =>  514831
cat stats =>  21159
journal stats =>  179132
year stats =>  4413
processed: 450000
aut stats =>  569994
cat stats =>  23098
journal stats =>  197556
year stats =>  4491
processed: 500000
aut stats =>  622476
cat stats =>  24888
journal stats

In [4]:
author_dict = sort_dict(author_dict)
cat_dict = sort_dict(cat_dict)
journal_dict = sort_dict(journal_dict)
year_dict = sort_dict(year_dict)

[autoreload of typing_extensions failed: Traceback (most recent call last):
  File "/data/zyang/miniconda3/envs/pt/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/data/zyang/miniconda3/envs/pt/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/data/zyang/miniconda3/envs/pt/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/data/zyang/miniconda3/envs/pt/lib/python3.9/site-packages/typing_extensions.py", line 733, in <module>
    class SupportsAbs(Protocol[T_co]):
  File "/data/zyang/miniconda3/envs/pt/lib/python3.9/typing.py", line 274, in inner
    return cached(*args,

In [5]:
# with open('author.pickle', 'wb') as handle:
#     pickle.dump(author_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('cat.pickle', 'wb') as handle:
#     pickle.dump(cat_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('journal.pickle', 'wb') as handle:
#     pickle.dump(journal_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('date.pickle', 'wb') as handle:
#     pickle.dump(year_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Filter data for all metadata frequencies > 1

In [6]:
chunks = pd.read_json(
    "../data/arxiv-metadata-oai-snapshot.json", lines=True, chunksize=50000
)
filtered_df = None
for i, chunk in enumerate(chunks):
    df = chunk
    df["cat_freq"] = df["categories"].map(cat_dict)
    df["journal_freq"] = df["journal-ref"].map(journal_dict)
    df["date_freq"] = df["update_date"].map(year_dict)
    df = df.loc[df["cat_freq"] > 1]
    print(df.shape)
    df = df.loc[df["journal_freq"] > 1]
    print(df.shape)
    df = df.loc[df["date_freq"] > 1]
    print(df.shape, "\n")
    if i == 0:
        filtered_df = df
    else:
        filtered_df = pd.concat([filtered_df, df])

(49411, 17)
(430, 17)
(430, 17) 

(49374, 17)
(292, 17)
(292, 17) 

(49324, 17)
(172, 17)
(172, 17) 

(49101, 17)
(283, 17)
(283, 17) 

(48923, 17)
(134, 17)
(133, 17) 

(48863, 17)
(97, 17)
(97, 17) 

(48949, 17)
(126, 17)
(126, 17) 

(48897, 17)
(83, 17)
(83, 17) 

(49045, 17)
(119, 17)
(119, 17) 

(49099, 17)
(96, 17)
(96, 17) 

(49096, 17)
(68, 17)
(68, 17) 

(49039, 17)
(55, 17)
(55, 17) 

(49035, 17)
(63, 17)
(63, 17) 

(49065, 17)
(80, 17)
(80, 17) 

(49068, 17)
(122, 17)
(122, 17) 

(49074, 17)
(165, 17)
(165, 17) 

(49000, 17)
(201, 17)
(201, 17) 

(48978, 17)
(195, 17)
(195, 17) 

(48903, 17)
(316, 17)
(316, 17) 

(48965, 17)
(371, 17)
(371, 17) 

(49035, 17)
(406, 17)
(406, 17) 

(49046, 17)
(500, 17)
(499, 17) 

(48841, 17)
(576, 17)
(576, 17) 

(48627, 17)
(526, 17)
(526, 17) 

(48742, 17)
(580, 17)
(580, 17) 

(48627, 17)
(539, 17)
(539, 17) 

(48702, 17)
(554, 17)
(554, 17) 

(48764, 17)
(577, 17)
(577, 17) 

(48707, 17)
(563, 17)
(563, 17) 

(48728, 17)
(553, 17)
(553, 

In [8]:
filtered_df.to_pickle("../data/filtered_data.pickle")